In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pickle
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pennylane as qml
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [5]:
test_data_path = 'test_air_quality_data.parquet'

In [6]:
n_qubits = 4
n_layers = 3

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dev = qml.device("lightning.qubit", wires=n_qubits)

In [8]:
@qml.qnode(dev, interface="torch")
def q_circuit(inputs, weights):
    """A quantum circuit that acts as a feature extractor."""
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [10]:
class QLSTMModel(nn.Module):
    """
    Hybrid Quantum-Classical model for multi-step forecasting.
    A classical LSTM processes the sequence, and its output is fed
    into a quantum circuit for feature extraction, followed by a
    classical layer for multi-step prediction.
    """
    def __init__(self, n_features, n_lstm_units=4, n_qubits=2, num_layers=1, n_layers=1, output_len=72):
        super(QLSTMModel, self).__init__()

        # 1. Classical LSTM Layer
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_lstm_units,
            num_layers=num_layers,
            batch_first=True
        )
        
        # 2. Classical Layer to map LSTM output to Quantum input
        self.classical_to_quantum = nn.Linear(n_lstm_units, n_qubits)
        
        # 3. Quantum Layer
        weight_shapes = {"weights": (n_layers, n_qubits, 3)}
        self.q_layer = qml.qnn.TorchLayer(q_circuit, weight_shapes)
        
        # 4. Classical Layer to map quantum output to predictions
        self.quantum_to_output = nn.Linear(n_qubits, output_len)
        
    def forward(self, x):        
        # 1. Pass data through the classical LSTM
        lstm_out, _ = self.lstm(x)
        
        # 2. Extract features from the last timestep
        final_lstm_output = lstm_out[:, -1, :]
        
        # 3. Prepare the data for the quantum circuit
        quantum_input = self.classical_to_quantum(final_lstm_output)
        
        # 4. Pass the features through the quantum circuit
        quantum_features = self.q_layer(quantum_input)
        
        # 5. Map quantum features to output sequence
        output = self.quantum_to_output(quantum_features)
        
        # 6. Apply sigmoid activation to get probabilities
        return torch.sigmoid(output)

In [11]:
def transform_data(file_path, lat_bins, lon_bins, le_location):
    """Transform raw test data using saved preprocessing objects"""
    df = pd.read_parquet(file_path)
    
    # Use provided bins from training data
    lat_positions = pd.cut(df['lat'], bins=lat_bins, labels=False, include_lowest=True)
    lon_positions = pd.cut(df['lon'], bins=lon_bins, labels=False, include_lowest=True)
    df['location'] = lat_positions * 20 + lon_positions
    df['location'] = df['location'].fillna(0).astype(int)
    
    # Process class and time features
    df['class'] = df['class'].isin(['Good', 'Moderate']).astype(int)
    df['time'] = pd.to_datetime(df['time'])
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month
    df['day'] = df['time'].dt.day
    df['hour'] = df['time'].dt.hour
    
    # Select columns
    list_of_columns = ['class', 'PM25_MERRA2', 'DUCMASS', 'TOTANGSTR', 'DUFLUXV', 'SSFLUXV', 'DUFLUXU', 'BCCMASS', 'SSSMASS25', 'location']
    selected_columns = list_of_columns + ['year', 'month', 'day', 'hour']
    df = df[selected_columns]
    
    # Encode location using saved encoder
    df['location_encoded'] = df['location'].map(
        lambda x: le_location.transform([x])[0] if x in le_location.classes_ else -1
    )
    
    df = df.sort_values(['location_encoded', 'year', 'month', 'day', 'hour'])
    
    # Define feature columns
    feature_columns = [col for col in df.columns if col not in [
        'class', 'location', 'year', 'month', 'day', 'hour', 'location_encoded'
    ]]
    df.drop(columns='location', inplace=True)
    feature_columns.append('location_encoded')
    
    return df, feature_columns

In [12]:
def create_sequences_memory_efficient(df, feature_columns, scaler, 
                                    input_len=168, output_len=72, stride=24):
    """Create sequences for testing using saved scaler"""
    print(f"Creating sequences with input length={input_len}, output length={output_len}...")
    
    X_scaled = scaler.transform(df[feature_columns])
    X_scaled = pd.DataFrame(X_scaled, columns=feature_columns, index=df.index)
    
    X_sequences, y_sequences, location_indices = [], [], []
    unique_locations = df['location_encoded'].unique()

    for i, loc in enumerate(unique_locations):
        loc_df = df[df['location_encoded'] == loc]
        loc_X = X_scaled.loc[loc_df.index]
        loc_y = loc_df['class'].values

        max_start_idx = len(loc_df) - input_len - output_len

        for j in range(0, max_start_idx, stride):
            X_seq = loc_X.iloc[j : j + input_len].values
            y_target = loc_y[j + input_len : j + input_len + output_len]

            X_sequences.append(X_seq)
            y_sequences.append(y_target)
            location_indices.append(loc)

        if (i+1) % 100 == 0:
            print(f"Processed location {i+1}/{len(unique_locations)}")

    X_sequences = np.array(X_sequences, dtype=np.float32)
    y_sequences = np.array(y_sequences, dtype=np.float32)
    location_indices = np.array(location_indices)

    print(f"Total sequences: {X_sequences.shape[0]}")
    print(f"Input sequence shape: {X_sequences.shape}")
    print(f"Output sequence shape: {y_sequences.shape}")

    return X_sequences, y_sequences, location_indices

In [13]:
def plot_example_predictions(model, X_test, y_test, num_examples=3):
    """Plot example predictions vs actual values"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    with torch.no_grad():
        sample_indices = np.random.choice(len(X_test), num_examples, replace=False)
        plt.figure(figsize=(15, 5 * num_examples))

        for i, idx in enumerate(sample_indices):
            X_sample = torch.tensor(X_test[idx:idx+1]).to(device)
            y_true = y_test[idx]
            y_pred = model(X_sample).cpu().numpy()[0]
            y_pred = (y_pred > 0.5).astype(int)

            plt.subplot(num_examples, 1, i+1)
            hours = np.arange(1, 72 + 1)
            plt.plot(hours, y_true, 'bo-', label='Actual')
            plt.plot(hours, y_pred, 'ro--', label='Predicted')
            plt.title(f'Example {i+1}: Air Quality Prediction (Next 72 Hours)')
            plt.xlabel('Hours Ahead')
            plt.ylabel('Air Quality Class')
            plt.yticks([0, 1], ['Good', 'Poor'])
            plt.legend()
            plt.grid(True)

        plt.tight_layout()
        plt.savefig('example_predictions.png', dpi=300, bbox_inches='tight')
        plt.show()

In [ ]:
def evaluate_model_per_location_and_hour(model, test_loader, device, location_indices_test, output_seq_len=72):
    """
    Evaluate the model with the needed metrics
    """
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            preds = (outputs > 0.5).float()

            all_preds.append(preds.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    preds_flat, labels_flat = all_preds.flatten(), all_labels.flatten()

    # All metrics Required
    accuracy = (preds_flat == labels_flat).mean()
    precision = precision_score(labels_flat, preds_flat, average="binary")
    recall = recall_score(labels_flat, preds_flat, average="binary")
    f1 = f1_score(labels_flat, preds_flat, average="binary")
    conf_matrix = confusion_matrix(labels_flat, preds_flat)

    # Per-location metrics
    unique_locations = np.unique(location_indices_test)
    location_metrics = {}
    for loc in unique_locations:
        idx = (location_indices_test == loc)
        preds_loc, labels_loc = all_preds[idx].flatten(), all_labels[idx].flatten()
        location_metrics[int(loc)] = {
            "accuracy": (preds_loc == labels_loc).mean(),
            "precision": precision_score(labels_loc, preds_loc, average="binary"),
            "recall": recall_score(labels_loc, preds_loc, average="binary"),
            "f1": f1_score(labels_loc, preds_loc, average="binary"),
            "confusion_matrix": confusion_matrix(labels_loc, preds_loc)
        }

    # Per-hour metrics
    hour_metrics = []
    for hour in range(output_seq_len):
        preds_hour, labels_hour = all_preds[:, hour], all_labels[:, hour]
        hour_metrics.append({
            "hour": hour + 1,
            "accuracy": (preds_hour == labels_hour).mean(),
            "precision": precision_score(labels_hour, preds_hour, average="binary"),
            "recall": recall_score(labels_hour, preds_hour, average="binary"),
            "f1": f1_score(labels_hour, preds_hour, average="binary"),
            "confusion_matrix": confusion_matrix(labels_hour, preds_hour)
        })

    return {
        "overall": {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "confusion_matrix": conf_matrix
        },
        "per_location": location_metrics,
        "per_hour": hour_metrics,
        "preds_flat": preds_flat,  
        "labels_flat": labels_flat  
    }

In [15]:
def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=[0,1], yticklabels=[0,1])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(title)
    plt.show()

In [16]:
if torch.cuda.is_available():
    print(f"✅ Found GPU: {torch.cuda.get_device_name(0)}. Using CUDA.")
else:
    print("❌ No GPU found. The script will run on the CPU.")

✅ Found GPU: NVIDIA GeForce RTX 3050 Laptop GPU. Using CUDA.


In [18]:
with open('preprocessing_state.pkl', 'rb') as f:
    preprocessing_state = pickle.load(f)

In [20]:
lat_bins = preprocessing_state['lat_bins']
lon_bins = preprocessing_state['lon_bins']
le_location = preprocessing_state['le_location']
feature_columns = preprocessing_state['feature_columns']
scaler = joblib.load('scaler.pkl')

df, feature_columns = transform_data(test_data_path, lat_bins, lon_bins, le_location)

X_test, y_test, location_indices = create_sequences_memory_efficient(
    df, feature_columns, scaler, input_len=168, output_len=72, stride=24
)

Creating sequences with input length=168, output length=72...
Processed location 100/400
Processed location 200/400
Processed location 300/400
Processed location 400/400
Total sequences: 79340
Input sequence shape: (79340, 168, 9)
Output sequence shape: (79340, 72)


In [21]:
batch_size = 512
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [22]:
n_lstm_units = 32
num_layers = 1

model = QLSTMModel(
    n_features=len(feature_columns), 
    n_lstm_units=n_lstm_units,  
    n_qubits=n_qubits,
    num_layers=num_layers,
    n_layers=n_layers,
    output_len=72
)

model.load_state_dict(torch.load('best_qlstm_model_multistep.pth'))
print("\nHybrid Quantum-Classical LSTM Model Architecture:")
print(model)

# Evaluate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Hybrid Quantum-Classical LSTM Model Architecture:
QLSTMModel(
  (lstm): LSTM(9, 32, batch_first=True)
  (classical_to_quantum): Linear(in_features=32, out_features=4, bias=True)
  (q_layer): <Quantum Torch Layer: func=q_circuit>
  (quantum_to_output): Linear(in_features=4, out_features=72, bias=True)
)


/tmp/ipykernel_17873/2255449281.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_qlstm_model_multistep.pth'))


QLSTMModel(
  (lstm): LSTM(9, 32, batch_first=True)
  (classical_to_quantum): Linear(in_features=32, out_features=4, bias=True)
  (q_layer): <Quantum Torch Layer: func=q_circuit>
  (quantum_to_output): Linear(in_features=4, out_features=72, bias=True)
)

In [24]:
print("\nCalculating detailed accuracy metrics...")
metrics  = evaluate_model_per_location_and_hour(
    model, test_loader, device, location_indices
)


Calculating detailed accuracy metrics...


KeyboardInterrupt: 

In [ ]:
overall_metrics = metrics["overall"]
location_metrics = metrics["per_location"]
hour_metrics = metrics["per_hour"]

In [ ]:
print("\nPer-location accuracy (averaged over all 72 hours):")
for loc in sorted(location_metrics.keys()):
    print(f"Location {loc}: {location_metrics[loc]['accuracy']:.4f}")

# Plot location accuracy distribution
plt.figure(figsize=(10, 6))
plt.hist([loc_metrics['accuracy'] for loc_metrics in location_metrics.values()], bins=20, edgecolor='black')
plt.title('Distribution of Per-Location Accuracy')
plt.xlabel('Accuracy')
plt.ylabel('Number of Locations')
plt.grid(True)
plt.savefig('location_accuracy_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("\nPer-hour accuracy (averaged over all locations):")
for hour_metrics_dict in hour_metrics:
    hour = hour_metrics_dict["hour"]
    accuracy = hour_metrics_dict["accuracy"]
    print(f"Hour {hour}: {accuracy:.4f}")

# Plot hour accuracy
plt.figure(figsize=(12, 6))
plt.plot([hm["hour"] for hm in hour_metrics], [hm["accuracy"] for hm in hour_metrics], marker='o')
plt.title('Accuracy per Forecast Hour')
plt.xlabel('Hour Ahead')
plt.ylabel('Accuracy')
plt.grid(True)
plt.savefig('hour_accuracy.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# After getting metrics
print("\nCalculating detailed accuracy metrics...")
metrics = evaluate_model_per_location_and_hour(
    model, test_loader, device, location_indices
)

# Extract metrics
overall_metrics = metrics["overall"]
location_metrics = metrics["per_location"]
hour_metrics = metrics["per_hour"]
preds_flat = metrics["preds_flat"]
labels_flat = metrics["labels_flat"]

# Plot overall confusion matrix
print("\nOverall Confusion Matrix:")
plt.figure(figsize=(8, 6))
sns.heatmap(overall_metrics['confusion_matrix'], 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=['Good', 'Poor'],
            yticklabels=['Good', 'Poor'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Overall Confusion Matrix')
plt.savefig('overall_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Print overall metrics
print("\nOverall Metrics:")
print(f"Accuracy: {overall_metrics['accuracy']:.4f}")
print(f"Precision: {overall_metrics['precision']:.4f}")
print(f"Recall: {overall_metrics['recall']:.4f}")
print(f"F1 Score: {overall_metrics['f1']:.4f}")

In [ ]:
print("\nGenerating example predictions...")
plot_example_predictions(model, X_test, y_test, num_examples=3)

print("\nTesting complete. Results saved.")